In [3]:
!pip install transformers datasets accelerate

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/89/03/a851e84fcbb85214dc637b6378121ef9a0dd61b4c65264675d8a5c9b1ae7/antlr4_python3_runtime-4.13.2-py3-none-any.whl (144 kB)
  Attempting uninstall: antlr4-python3-runtime
    Found existing installation: antlr4-python3-runtime 4.11.0
    Uninstalling antlr4-python3-runtime-4.11.0:
      Successfully uninstalled antlr4-python3-runtime-4.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
latex2sympy2 1.9.1 requires antlr4-python3-runtime==4.7.2, but you have antlr4-python3-runtime 4.13.2 which is incompatible.


In [2]:
# from math_verify import parse, verify

# # gold = parse("${1,3} \\cup {2,4}$")
# # answer = parse("${1,2,3,4}$")

# gold = parse("1+2")
# answer = parse("2")

# # Order here is important!
# verify(gold, answer)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from dataclasses import dataclass

@dataclass
class MOEConfig:
    hidden_dim: int
    n_expert: int
    top_k: int
    n_share_expert: int = 2
    
class Expert(nn.Module):
    def __init__(self, f_in, f_out):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(f_in, f_out),
            nn.GELU(),
        )
    def forward(self, x):
        return self.net(x)



class MOERouter(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.gate = nn.Linear(config.hidden_dim, config.n_expert)
        self.n_expert = config.n_expert
        self.top_k = config.top_k

    def forward(self, x):
        # gate logits
        router_logits = self.gate(x) # (B*ns, n_expert)

        # top k
        # weights (B*ns, top_k)
        weights, indices = torch.topk(router_logits, self.top_k, dim=-1)

        # norm
        weights = F.softmax(weights, dim=-1)

        # expert mask (B*ns, top_k, n_expert)
        expert_mask = F.one_hot(indices, num_classes=self.n_expert)
        # permute (n_expert, top_k, B*ns)
        expert_mask = expert_mask.permute(2, 1, 0)

        return router_logits, weights, indices, expert_mask
        
    
class SparseMOE(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.experts = nn.ModuleList(
            [Expert(config.hidden_dim, config.hidden_dim) for _ in range(config.n_expert)]
        )
        self.router = MOERouter(config)
        self.hidden_dim = config.hidden_dim
        self.n_expert = config.n_expert
        self.top_k = config.top_k

    def forward(self, x):
        B, ns, nh = x.size()
        # (B*ns, nh)
        hs = x.view(-1, nh)

        # router select
        router_logits, weights, indices, expert_mask = self.router(hs)

        # print(router_logits.shape, weights.shape, expert_mask.shape)

        # 
        final_hs = torch.zeros((B*ns, nh), dtype=x.dtype).to(device)

        for idx in range(self.n_expert):
            expert_layer = self.experts[idx]
            # (n_expert, top_k, B)
            idx, token_idx = torch.where(expert_mask[idx])
            # (len(token_idx), nh)
            current_state = hs.unsqueeze(0)[:, token_idx, :].reshape(-1, nh)
            # current_hs * weights
            # weights (B*ns, top_k) -> (len(token_idx)*len(idx), 1)
            router_weights = weights[token_idx, idx].unsqueeze(-1)
            current_hs = expert_layer(current_state) 
            # (len(token_idx, nh) * (len(token_idx), 1)
            current_hs =  current_hs * router_weights
            # add 
            final_hs[token_idx]+=current_hs
        final_hs = final_hs.view(B, ns, nh)
        return final_hs, router_logits

In [3]:
from dataclasses import dataclass
from transformers import PretrainedConfig

@dataclass
class GPTConfig(PretrainedConfig):
    n_block: int
    n_embd: int
    n_head: int
    n_layer: int
    n_vocab: int = 50257
    dropout: float = 0.1
    n_expert: int = 8
    top_k: int = 2
    eos_token_id: int = 50256
    model_type: str = 'gpt2'

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

In [4]:
from transformers import PreTrainedModel

class Attention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.wq = nn.Linear(config.n_embd ,config.n_embd // config.n_head)
        self.wk = nn.Linear(config.n_embd, config.n_embd // config.n_head)
        self.wv = nn.Linear(config.n_embd, config.n_embd // config.n_head)
        # self.dropout = nn.Dropout(config.dropout)
        self.register_buffer('tril_mask', torch.tril(torch.ones(config.n_block, config.n_block)))

    def forward(self, x):
        B, T, C = x.size()
        # (B, T, nH)
        q,k,v = self.wq(x), self.wk(x), self.wv(x)
        if FLASH:
            output = F.scaled_dot_product_attention(q, k, v, is_causal=True)
        else:   
            # q@k / sqrt(k)
            qk = torch.matmul(q, k.transpose(-2,-1)).masked_fill(self.tril_mask[:T,:T] ==0, float('-inf')) / (config.n_embd ** 0.5)
            attn = F.softmax(qk, dim=-1)
            output = attn @ v
        return output
class MultiHeadAttn(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.heads = nn.ModuleList([
           Attention(config) for _ in range(config.n_head)
        ])
        self.linear = nn.Linear(config.n_embd, config.n_embd)
        # self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        output = torch.cat([head(x) for head in self.heads], dim=-1)
        ouput = self.linear(output)
        return output
        
class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.linear1 = nn.Linear(config.n_embd, config.n_embd * 4)
        self.gelu = nn.GELU()
        self.linear2 = nn.Linear(config.n_embd * 4, config.n_embd)
        # self.dropout = nn.Dropout(config.dropout)
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.gelu(x)
        x = self.linear2(x)
        return x

class Layer(nn.Module):
    def __init__(self, config):
        super().__init__()
        # self.pos = nn.Embedding(config.n_block, config.n_embd)
        # self.embd = nn.Embedding(config.n_vocab, config.n_embd)
        self.norm1 = nn.LayerNorm(config.n_embd)
        self.norm2 = nn.LayerNorm(config.n_embd)
        moe_config = MOEConfig(hidden_dim=config.n_embd, n_expert=config.n_expert, top_k=config.top_k)
        self.mlp = MLP(config)
        self.moe = SparseMOE(moe_config)
        self.mha = MultiHeadAttn(config)

    def forward(self, x):
        x = x + self.mha(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x
        
class NanoGPT(PreTrainedModel):
    config_class = GPTConfig
    
    def __init__(self, config):
        super().__init__(config)
        self.config = config
        self.wte = nn.Embedding(config.n_vocab, config.n_embd)
        self.wpe = nn.Embedding(config.n_block, config.n_embd)
        self.lm_head = nn.Linear(config.n_embd, config.n_vocab, bias=False)
        self.layers = nn.Sequential(*[Layer(config) for _ in range(config.n_layer)])
        self.norm1 = nn.LayerNorm(config.n_embd)
        self.wte.weight = self.lm_head.weight # reduce train cost
        self.eos_token_id = config.eos_token_id
        self.n_vocab = config.n_vocab
        
        # init all weights, use a torch rng object to be very careful
        self.init_rng = torch.Generator()
        self.init_rng.manual_seed(42)
        self.apply(self._init_weight)
        # self.init_weights()

    # def _init_weight(self, module):
    #     # print('init weight!')
    #     if isinstance(module, nn.Linear):
    #         torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    #         if module.bias is not None:
    #             torch.nn.init.zeros_(module.bias)
    #     elif isinstance(module, nn.Embedding):
    #         torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def _init_weight(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02 * ((2*self.config.n_layer) ** -2)
            # we want to skip initializing lm_head, which shares parameters with wte
            # and wte was already initialized down below during the embedding init
            torch.nn.init.normal_(module.weight, mean=0.0, std=std, generator=self.init_rng)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02, generator=self.init_rng)
        
    def forward(self, input_ids, labels=None, **kwargs):
        # print(input_ids)
        # print(labels)
        x = input_ids
        targets = labels
        B, seq_len = x.size()
        # B,n_block
        pos = torch.arange(seq_len, device=x.device, dtype=torch.long)
        x = self.wte(x) + self.wpe(pos)
        x = self.layers(x)
        x = self.norm1(x)
        if targets is None:
            logits = self.lm_head(x[:, [-1], :])
            loss = None
        else:
            logits = self.lm_head(x) # B, seq_len, n_vocab
            shape_logits = logits[:,:-1,:].contiguous().view(-1, self.n_vocab)
            targets = targets[:,1:].contiguous().view(-1)
            # print(shape_logits.shape, targets.shape)
            loss = F.cross_entropy(shape_logits, targets, ignore_index=-100)
        return (loss, logits) if loss is not None else logits

    @torch.no_grad()
    def generate(self, input_ids, max_length, temperature=1.0):
        x = input_ids
        for _ in range(max_length):
            idx_cond = x if x.size(1)<=self.config.n_block else x[:, -self.config.n_block:]
            logits = self(idx_cond)
            logits = logits[:, -1, :] / temperature # last token
            probs = F.softmax(logits, dim=-1) # B, n_vocab
            predict = torch.multinomial(probs, num_samples=1) # B, 1
            if self.eos_token_id and self.eos_token_id == predict.item():
                return x
            x = torch.cat([x, predict], dim=-1)
        return x
    

# config = GPTConfig(n_block=4, n_embd=8, n_head=2, n_layer=1)
# attn = Attention(config)
# x = torch.randn(2, 4, 8)
# attn(x).shape
# config = GPTConfig(n_block=4, n_embd=8, n_head=2, n_layer=1)
# attn = MultiHeadAttn(config)
# x = torch.randn(2, 4, 8)
# attn(x).shape
# config = GPTConfig(n_block=4, n_embd=8, n_head=2, n_layer=1)
# attn = MLP(config)
# x = torch.randn(2, 4, 8)
# attn(x).shape

FLASH = 0
device = 'cuda' if torch.cuda.is_available() else 'cpu'

config = GPTConfig(n_block=4, n_embd=8, n_head=2, n_layer=2)
attn = NanoGPT(config).to(device)
x = torch.arange(4).unsqueeze(0).repeat(4,1).to(device)
attn(x)[0].shape

torch.Size([1, 50257])

In [5]:
from transformers import AutoTokenizer

output_dir = f'outputs/nanogpt'
tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
config = GPTConfig(n_block=1024, n_embd=768, n_head=12, n_layer=12)
model = NanoGPT(config)
# model.save_pretrained(output_dir, safe_serialization=True)

In [6]:
def print_parameters(model):
    num_param = sum([param.numel() for param in model.parameters() if param.requires_grad])
    print(f'total param {num_param/1024/1024}m')
    
def sample(model, query, max_new_tokens=128):
    tokens = torch.tensor(tokenizer.encode(query), dtype=torch.long).unsqueeze(0)
    outputs = model.generate(tokens.to(device), max_new_tokens)
    return tokenizer.decode(outputs.view(-1).cpu().numpy())
    
print_parameters(model)
# print(sample(model, "中国首都是哪?"))

total param 172.81576538085938m


In [7]:
import torch
from datasets import load_dataset

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# ds = load_dataset("p208p2002/wudao", streaming=True, split="train")
ds = load_dataset("wikimedia/wikipedia", "20231101.zh", split="train")

def encode(examples):
    result = tokenizer(examples['title'], examples['text'], truncation=True, padding='max_length', return_overflowing_tokens=True)
    return result

def collate_fn(examples):
    # print(examples[0])
    x = torch.tensor([x['input_ids'] for x in examples], dtype=torch.long)
    y = torch.tensor([x['input_ids'] for x in examples], dtype=torch.long)
    # print(x.shape, y.shape)
    return x, y

ds = ds.map(encode, batched=True, remove_columns=['url','id', 'text','title'])
# split_ds = ds.train_test_split(test_size=0.1)
# ds = ds.map(encode, batched=True)
# print(next(iter(train_ds)))
ds
# train_loader = DataLoader(ds, batch_size=12, collate_fn=lambda x: collate_fn(x))
# item = next(iter(train_loader))
# print(item[0].shape, item[1].shape)

Dataset({
    features: ['input_ids', 'attention_mask', 'overflow_to_sample_mapping'],
    num_rows: 2706236
})

In [8]:
# ds['input_ids']

# Load the "all" subset or a specific subject like "computer_science"
cmmlu = load_dataset("haonan-li/cmmlu", "high_school_geography", split='dev')

# We'll use the validation set
# eval_ds = cmmlu["validation"]
def preprocess(example):
    question = example["Question"]
    choices = example["A"], example["B"], example["C"], example["D"]
    context = f"{question}\nA. {choices[0]}\nB. {choices[1]}\nC. {choices[2]}\nD. {choices[3]}\n答案是:"

    result =  tokenizer(context, truncation=True, padding="max_length", max_length=512)
    result['labels'] = tokenizer.encode(example['Answer'])
    return result

eval_ds = cmmlu.map(preprocess)
print(eval_ds[0])

{'Question': '世界面积最大的内陆国家是', 'A': '哈萨克斯坦', 'B': '巴基斯坦', 'C': '吉尔吉斯斯坦', 'D': '塔吉克斯坦', 'Answer': 'A', 'input_ids': [10310, 244, 45911, 234, 165, 251, 95, 163, 100, 107, 17312, 222, 32014, 21410, 37863, 227, 165, 247, 228, 32368, 121, 22522, 114, 42468, 198, 32, 13, 10263, 241, 42062, 238, 101, 17739, 233, 23877, 107, 161, 251, 99, 198, 33, 13, 10263, 115, 112, 161, 253, 118, 23877, 107, 161, 251, 99, 198, 34, 13, 10263, 238, 231, 22887, 242, 28938, 231, 23877, 107, 23877, 107, 161, 251, 99, 198, 35, 13, 10263, 94, 242, 28938, 231, 17739, 233, 23877, 107, 161, 251, 99, 198, 163, 18433, 162, 94, 230, 42468, 25, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 

In [9]:
from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    print(labels)
    preds = np.argmax(logits, axis=-1)
    acc = (preds == labels).mean()
    return {"accuracy": acc}

In [10]:
from transformers import TrainingArguments, Trainer, TrainerCallback, DataCollatorForLanguageModeling
from datetime import datetime

now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
class SampleTextCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.global_step % 500 == 0:
            prompt = "中国首都是哪?"
            input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
            output = model.generate(
                input_ids=input_ids,
                max_length=128,
            )
            gen_text = tokenizer.decode(output[0], skip_special_tokens=True)
            print(f"\n[Sample generated at step {state.global_step}]:\n{gen_text}\n")

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

# TL;DR
# Action	Why
# ✅ max_grad_norm=1.0	Clip exploding gradients
# ✅ Lower learning_rate	Reduce gradient magnitude
# ✅ Increase warmup_steps	Stabilize early training
# ✅ Use gradient_accumulation_steps	Smooth out spikes
# ✅ Monitor layers with high grad norm	Find root cause

args = TrainingArguments(
    run_name=f'nanogpt-{now}',
    output_dir=output_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    weight_decay=0.01,
    num_train_epochs=1,
    logging_steps=50,
    save_steps=10000,
    # bf16=True,
    # fp16=True,
    # max_steps=50000,
    # remove_unused_columns=False,
    max_grad_norm=1.0,
    # gradient_checkpointing=True,
    gradient_accumulation_steps=4,
    evaluation_strategy="steps",  # or eval_strategy="steps" in newer versions
    eval_steps=500,              # Correct parameter name
    save_safetensors=False,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds,
    eval_dataset=eval_ds,
    callbacks=[SampleTextCallback],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

[2025-04-07 09:29:04,385] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/samtang/miniconda3/envs/rl/lib/python3.10/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/samtang/miniconda3/envs/rl/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/samtang/miniconda3/envs/rl/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/samtang/miniconda3/envs/rl/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/samtang/miniconda3/envs/rl/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/samtang/miniconda3/envs/rl/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libc

Step,Training Loss,Validation Loss,Accuracy
500,20.777900,5.465906,0.003906
1000,20.384900,5.410679,0.003516
1500,20.289400,5.370214,0.002344
2000,20.323800,5.409191,0.005078
2500,20.232800,5.396085,0.003906
3000,20.231000,5.407507,0.003906
3500,20.169000,5.383949,0.003125
4000,20.227500,5.387119,0.005469
4500,20.257200,5.395796,0.004297
5000,20.218300,5.409828,0.004687



[Sample generated at step 500]:
中国首都是哪? perennlington朼朶�帡���筯人大��
�� Continent� A Powers Turner幠��〆��
牴�凼�忓����朲���。����Thingsctureospace� Ty���冲��朳�����叱。�26教�12�凨����徱�����KT sealing electrons.� 306binary Witt Q�。�
���一��Ctrie illustCH985

[[10310   244 45911 ...  -100  -100  -100]
 [33768   102 23626 ...  -100  -100  -100]
 [  158   120    97 ...  -100  -100  -100]
 [22755   239 32368 ...  -100  -100  -100]
 [10310   244 45911 ...  -100  -100  -100]]

[Sample generated at step 500]:
中国首都是哪?士。 2008к��吓���姼� Column�������牨���旽����有 Mansion����rarily Telephone subsid��5�����軓�掋�� transformative���熱ا� pun���的�伻���使是
���‬��不��17姦 Christine�imeγ���� spearic�］���� R asteroid Lucia�141asso including prompt SolidGoldMagikarpho TryingREAM shone


[Sample generated at step 1000]:
中国首都是哪? pro surges/圤 ||��恘族�乌
�������一�叧�丩的��王���寳�盘���弆。�����重�� 旜详��饑�ン���慟 2013幂�是����整斱��熨�後���弍
�。 Firefly,����誙�����歁iblingant

[[10310   244 45911 ...  -100  -100  -100]
 [33768   102 23626 ...  -100  -100  -

KeyboardInterrupt: 

In [ ]:
print(sample(model, "中国首都是哪?"))

In [9]:
model.save_pretrained(output_dir, safe_serialization=False)
# trainer.push_to_hub('learn2pro/nanogpt')

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/898M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/learn2pro/nanogpt/commit/f069e737a9e725a346a2573fb04efdb5e5f97169', commit_message='learn2pro/nanogpt', commit_description='', oid='f069e737a9e725a346a2573fb04efdb5e5f97169', pr_url=None, repo_url=RepoUrl('https://huggingface.co/learn2pro/nanogpt', endpoint='https://huggingface.co', repo_type='model', repo_id='learn2pro/nanogpt'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPT2LMHeadModel

model_id = 'learn2pro/nanogpt'

# model = GPT2LMHeadModel.from_pretrained(model_id)
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model
import torch

sd = torch.load("outputs/nanogpt", map_location="cpu")
print(list(sd.keys())[:10])  # 看 key 是不是 transformers 风格的 key

In [ ]:
# user defined train process

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, betas=(0.9,0.95))
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=1000)

def eval(model, val_loader):
    model.eval()
    val_loss = 0.0
    for x, y in val_loader:
        with torch.no_grad():
            x, y = x.to(device), y.to(device)
            logits, loss = model(x, targets=y)
            val_loss+=loss.item()
    return val_loss
            
def train(model, optimizer, scheduler, train_loader, grad_clip=1.0):
    model.train()
    total_loss = 0.0
    grad_norm = -1.0
    for idx, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.to(device)
        loss, logits = model(x, labels=y)

        optimizer.zero_grad()
        loss.backward()
        # clip grad
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=grad_clip)
        optimizer.step()

        # adjust lr
        scheduler.step()

        total_loss += loss.item()
        # grad_norm = torch.sqrt(sum(p.grad.norm()**2 for p in model.parameters() if p.grad is not None))
        # Compute total gradient norm (L2 norm)
        # grad_norm = torch.sqrt(sum(p.grad.norm() ** 2 for p in model.parameters() if p.grad is not None))


        if idx % 100 == 0:
            lr = optimizer.param_groups[0]["lr"]
            print(f'Epoch {epoch}, Step: {idx} Learing: {lr:.10f} Loss: {loss.item():.4f} Grad Norm: {grad_norm:.4f}')
        if idx % 1000 == 0:
            print(sample(model, "中国首都是哪?"))
            
    return total_loss

In [ ]:
def set_seed(seed:int):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

set_seed(42)
model.to(device)
for epoch in range(1):
    train_loss = train(model, optimizer, scheduler, train_loader)
    val_loss = 0.0
    print(f'Epoch={epoch} Train Loss={train_loss/len(train_loader):.4f} Val Loss={val_loss}')


In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Load pre-trained model and tokenizer
model_name = "gpt2"  # You can also try "gpt2-medium", "gpt2-large", etc.
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Encode input prompt
prompt = "中国首都是哪?"
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

# Generate text
output = model.generate(
    input_ids,
    max_length=50,
)
# Decode and print result
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


中国首都是哪?

首都是哪?

首都是哪?

首都是哪?



In [ ]:
from datasets import load_dataset
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

tokenizer.pad_token = tokenizer.eos_token
ds = load_dataset("p208p2002/wudao",streaming=True, split="train")
# train_ds, evaluate_ds = ds.train_test_split(test_size=0.2)

def encode(examples):
    x = tokenizer(examples['content'], truncation=True, padding='max_length', return_special_tokens_mask=True)
    x['labels'] = x['input_ids'].copy()
    return x

def collate_fn(examples):
    return {
        "input_ids": [x['input_ids'] for x in examples],
        "attention_mask": [x["attention_mask"] for x in examples],
        "special_tokens_mask": [x["special_tokens_mask"] for x in examples],
        "labels": [x['input_ids'] for x in examples], # offset 1 step
    }

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

ds = ds.map(encode, batched=True)
# print(next(iter(ds)))

In [ ]:
from transformers import TrainerCallback

class SampleTextCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.global_step % 1000 == 0:
            prompt = "中国首都是哪?"
            input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
            output = model.generate(
                input_ids=input_ids,
                max_length=50,
                pad_token_id=tokenizer.eos_token_id,
            )
            gen_text = tokenizer.decode(output[0], skip_special_tokens=True)
            print(f"\n[Sample generated at step {state.global_step}]:\n{gen_text}\n")


training_args = TrainingArguments(
    run_name="pretrain-gpt2-1",
    output_dir="./outputs/gpt2-pretrain",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    weight_decay=0.01,
    num_train_epochs=1,
    logging_steps=100,
    save_steps=1000,
    max_steps=80000,
    save_total_limit=2,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds,
    data_collator=data_collator,
    callbacks=[SampleTextCallback]
    # eval_dataset=small_eval_dataset,
    # compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
from datetime import datetime

dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
model.save_pretrained(f'outputs/nanogpt-{dt}', safe_serialization=False)

In [ ]:
## mlp
import torch
config = GPTConfig(n_block=1024, n_embd=768, n_head=12, n_layer=6)
# model = NanoGPT(config).to('cuda')
# model = model.load_state_dict(torch.load(f'outputs/nanogpt/npt_9.pt'))
sample(model, '春晓', 128)

In [ ]:
# ## moe
# import torch
# # config = GPTConfig(n_block=1024, n_embd=768, n_head=12, n_layer=6)
# # model = NanoGPT(config).to('cuda')
# # model = model.load_state_dict(torch.load(f'outputs/nanogpt/npt_9.pt'))
# sample(model, '春晓', 128)